<a href="https://colab.research.google.com/github/nikhiljose7/captionToImage/blob/main/caption_to_image_Backend.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# backend.py
# --- Step 1: Install necessary libraries ---
!pip install fastapi uvicorn pyngrok diffusers transformers accelerate torch Pillow

import torch
from fastapi import FastAPI
from pydantic import BaseModel
from diffusers import StableDiffusionPipeline
import base64
from io import BytesIO
from PIL import Image
import uvicorn
from pyngrok import ngrok, conf
import threading
import os

# --- Optional: Add your ngrok authtoken if you have one ---
# Replace "YOUR_NGROK_AUTHTOKEN" with your actual token from https://dashboard.ngrok.com/get-started/your-authtoken
conf.get_default().auth_token = "31QAivzbppt5jb8ozM3TkemWwlN_6BKgSbrv5qYALrgK7pZ3Q"

print("Starting backend server...")

# ---- Step 2: Load the Stable Diffusion model AND your LoRA ----
print("Loading Stable Diffusion model...")
# Define the base model and your LoRA model from the Hugging Face Hub
base_model_id = "runwayml/stable-diffusion-v1-5"
lora_model_id = "nikhiljose7/lora-sdv1-5-minnal-murali"

# Use float16 for faster inference and less memory usage.
pipe = StableDiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)

print(f"Loading LoRA weights from: {lora_model_id}")
# Load the LoRA weights into the base model
pipe.load_lora_weights(lora_model_id)

# Move the model to the GPU if available.
if torch.cuda.is_available():
    print("Moving model to GPU...")
    pipe = pipe.to("cuda")
else:
    print("CUDA not available. Using CPU (this will be very slow).")
    pipe = pipe.to("cpu")

# ---- Step 3: Set up the FastAPI app ----
app = FastAPI()

# Define the request body structure.
# The key must be "caption".
class Prompt(BaseModel):
    caption: str

@app.get("/")
def read_root():
    return {"status": "ok"}

# Define the image generation endpoint.
@app.post("/generate")
def generate_image(data: Prompt):
    """
    Generates an image from a text caption.
    """
    # --- MODIFIED: Add the trigger word to the prompt ---
    # This is crucial to activate your LoRA model's concept.
    trigger_word = "Minnal Murali"
    prompt_with_trigger = f"{trigger_word}, {data.caption}"

    print(f"Received caption: {data.caption}")
    print(f"Using full prompt: {prompt_with_trigger}")

    try:
        # Generate the image using the modified prompt.
        image = pipe(prompt_with_trigger).images[0]

        # Convert the PIL image to a base64 string to send via JSON.
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

        print("Image generated successfully.")
        return {"image_base64": img_str}
    except Exception as e:
        print(f"An error occurred: {e}")
        return {"error": str(e)}, 500


# ---- Step 4: Run the server using Uvicorn and Ngrok ----
# We run Uvicorn in a separate thread so that we can start ngrok in the main thread.
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Open a public tunnel to the local server
public_url = ngrok.connect(8000)
print("------------------------------------------------")
print(f"✅ Public API URL: {public_url}")
print("------------------------------------------------")
print("Copy this URL and paste it into the API_URL variable in your frontend.py script.")


# Start the FastAPI server in the background.
thread = threading.Thread(target=run_app, daemon=True)
thread.start()

# Keep the main thread alive to keep ngrok running.
try:
    while True:
        pass
except KeyboardInterrupt:
    print("Shutting down server and ngrok tunnel.")
    ngrok.disconnect(public_url)

Starting backend server...
Loading Stable Diffusion model...


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Loading LoRA weights from: nikhiljose7/lora-sdv1-5-minnal-murali


No LoRA keys associated to CLIPTextModel found with the prefix='text_encoder'. This is safe to ignore if LoRA state dict didn't originally have any CLIPTextModel related params. You can also try specifying `prefix=None` to resolve the warning. Otherwise, open an issue if you think it's unexpected: https://github.com/huggingface/diffusers/issues/new


Moving model to GPU...
------------------------------------------------
✅ Public API URL: NgrokTunnel: "https://a612a2f89238.ngrok-free.app" -> "http://localhost:8000"
------------------------------------------------
Copy this URL and paste it into the API_URL variable in your frontend.py script.


INFO:     Started server process [2078]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
ERROR:    [Errno 98] error while attempting to bind on address ('0.0.0.0', 8000): address already in use
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
